In [8]:

import itertools
import time
import math
import sys
from getpass import getpass

PRESETS = {
    "1": ("lowercase letters", "abcdefghijklmnopqrstuvwxyz"),
    "2": ("lower + digits", "abcdefghijklmnopqrstuvwxyz0123456789"),
    "3": ("lower+upper+digits", "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"),
    "4": ("lower+upper+digits+symbols", "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*()-_+=[]{};:,.<>?/\\|`~"),
    "5": ("digits only", "0123456789"),
}

SAFETY_MAX_COMBINATIONS = 200_000_000  # 200 million default safety cap
DEFAULT_PROGRESS_INTERVAL = 200_000    # print progress every N attempts

def candidate_generator_fixed_length(charset: str, length: int):
    """Yield only candidates of exactly the given length."""
    for tup in itertools.product(charset, repeat=length):
        yield ''.join(tup)


def estimate_total_fixed(charset_size: int, length: int) -> int:
    return pow(charset_size, length)



def fmt_int(n: int) -> str:
    return f"{n:,}"

def main():
    print("Educational brute-force simulator (only test your own passwords)\n")

    # get the target password securely
    target = getpass("Enter target password (hidden): ").strip()
    if not target:
        print("No password entered — exiting.")
        return

    # choose preset
    print("\nChoose a charset preset:")
    for key, (name, cs) in PRESETS.items():
        print(f"  {key}. {name} (size {len(cs)})")
    print("  6. custom charset")
    choice = input("Enter choice [default 2]: ").strip() or "2"
    if choice == "6":
        charset = input("Enter custom charset (characters to use): ").strip()
        if not charset:
            print("Empty charset — exiting.")
            return
    else:
        if choice not in PRESETS:
            print("Invalid choice — using default (lower+digits).")
            charset = PRESETS["2"][1]
        else:
            charset = PRESETS[choice][1]

    charset = "".join(dict.fromkeys(charset))  # remove duplicates, keep order
    cs_size = len(charset)
    if cs_size == 0:
        print("Charset is empty — exiting.")
        return

    # pick max length
    default_max_len = len(target)
    try:
        raw = input(f"Max length to try [default {default_max_len}]: ").strip()
        if raw == "":
            max_len = default_max_len
        else:
            max_len = int(raw)
            if max_len <= 0:
                print("Max length must be >=1. Exiting.")
                return
    except ValueError:
        print("Invalid number — exiting.")
        return

    # safety cap: compute total combos and warn/limit if too big
    try:
        total_candidates = estimate_total_fixed(cs_size, max_len)

    except OverflowError:
        total_candidates = float("inf")

    print("\nConfiguration:")
    print(f"  Charset size: {cs_size}")
    print(f"  Max length: {max_len}")
    print(f"  Estimated total candidates: {fmt_int(total_candidates) if total_candidates != float('inf') else 'infinite/too large'}")

    if total_candidates > SAFETY_MAX_COMBINATIONS:
        print(f"\nWARNING: estimated search space ({fmt_int(total_candidates)}) exceeds safety cap ({fmt_int(SAFETY_MAX_COMBINATIONS)}).")
        confirm = input("Type 'YES' to continue anyway, or anything else to abort: ")
        if confirm != "YES":
            print("Aborted by user (safety).")
            return

    progress_interval = DEFAULT_PROGRESS_INTERVAL
    try:
        raw = input(f"Progress print interval (attempts) [default {DEFAULT_PROGRESS_INTERVAL}]: ").strip()
        if raw:
            progress_interval = max(1, int(raw))
    except ValueError:
        pass

    print("\nStarting brute-force... (Ctrl+C to stop)\n")
    attempts = 0
    start_time = time.perf_counter()
    last_report_time = start_time
    last_attempts = 0

    try:
        for candidate in candidate_generator_fixed_length(charset, max_len):

            attempts += 1

            if attempts % progress_interval == 0:
                now = time.perf_counter()
                elapsed = now - start_time
                attempts_since = attempts - last_attempts
                last_attempts = attempts
                avg_attempt_time = elapsed / attempts if attempts else 0.0
                # ETA calculation (if total known)
                if total_candidates != float('inf'):
                    remaining = max(0, total_candidates - attempts)
                    eta_seconds = remaining * avg_attempt_time
                    # simple human-friendly ETA
                    def human_secs(s):
                        if s < 60:
                            return f"{s:.1f}s"
                        if s < 3600:
                            return f"{s/60:.1f}m"
                        return f"{s/3600:.1f}h"
                    eta_str = human_secs(eta_seconds)
                else:
                    eta_str = "unknown"

                print(f"[{fmt_int(attempts)}] elapsed {elapsed:.2f}s — avg {avg_attempt_time*1e3:.4f} ms/attempt — ETA {eta_str} — last tried: {candidate}")

            if candidate == target:
                elapsed = time.perf_counter() - start_time
                print("\n=== FOUND ===")
                print(f"Password: '{candidate}'")
                print(f"Attempts: {fmt_int(attempts)}")
                print(f"Elapsed time: {elapsed:.6f} seconds")
                if attempts > 0:
                    print(f"Avg time per attempt: {elapsed/attempts*1e3:.6f} ms")
                return

    except KeyboardInterrupt:
        elapsed = time.perf_counter() - start_time
        print("\nInterrupted by user (Ctrl+C).")
        print(f"Attempts so far: {fmt_int(attempts)} — elapsed {elapsed:.2f}s — avg {(elapsed/attempts*1e3) if attempts else 0:.6f} ms/attempt")
        return

    elapsed = time.perf_counter() - start_time
    print("\nCompleted search up to requested max length.")
    print(f"Password NOT found after {fmt_int(attempts)} attempts in {elapsed:.6f} seconds.")

if __name__ == "__main__":
    main()


Educational brute-force simulator (only test your own passwords)

Enter target password (hidden): ··········

Choose a charset preset:
  1. lowercase letters (size 26)
  2. lower + digits (size 36)
  3. lower+upper+digits (size 62)
  4. lower+upper+digits+symbols (size 92)
  5. digits only (size 10)
  6. custom charset
Enter choice [default 2]: 1
Max length to try [default 6]: 

Configuration:
  Charset size: 26
  Max length: 6
  Estimated total candidates: 308,915,776

Type 'YES' to continue anyway, or anything else to abort: YES
Progress print interval (attempts) [default 200000]: 50000000

Starting brute-force... (Ctrl+C to stop)


=== FOUND ===
Password: 'chitta'
Attempts: 27,115,531
Elapsed time: 6.401189 seconds
Avg time per attempt: 0.000236 ms
